#Group C | Part 0 - Data Selection

link to yelp documentation: https://www.yelp.com/dataset/download

In [1]:
# Install gdown if not already installed
!pip install --quiet gdown

In [3]:
from google.colab import drive, files
import os
import numpy as np
import cv2
import glob
import pandas as pd
import json
from matplotlib import pyplot as plt
import torch
import gdown
import traceback

In [3]:
# Execution device
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print("The model will be running on", device, "device")

The model will be running on cpu device


In [4]:
def download_and_extract_tar(file_id, folder_name='data', downloaded_file='data.tar'):
    """
    Downloads a .tar file from Google Drive, extracts it to a specified folder,
    and deletes the .tar file after extraction.

    Parameters:
      - file_id (str): The file ID of the .tar file on Google Drive.
      - folder_name (str): The folder where the .tar content will be extracted. Defaults to 'data'.
      - downloaded_file (str): The name to save the downloaded .tar file as. Defaults to 'data.tar'.
    """

    # Download the file from Google Drive using gdown
    gdown.download(f'https://drive.google.com/uc?export=download&id={file_id}', downloaded_file, quiet=False)

    # Create a folder to extract the files
    os.makedirs(folder_name, exist_ok=True)

    # Extract the .tar file into the folder
    os.system(f'tar -xf {downloaded_file} -C {folder_name}')

    # List the extracted files to verify
    print(f"Files extracted to '{folder_name}':")
    os.system(f'ls {folder_name}')

    # Remove the .tar file
    if os.path.exists(downloaded_file):
        os.remove(downloaded_file)
        print(f"{downloaded_file} deleted successfully.")
    else:
        print(f"{downloaded_file} does not exist.")


In [5]:
file_id = '1-ZciGB5VJK544sZCafYaukbjsHDCQagH'
download_and_extract_tar(file_id)

Downloading...
From (original): https://drive.google.com/uc?export=download&id=1-ZciGB5VJK544sZCafYaukbjsHDCQagH
From (redirected): https://drive.google.com/uc?export=download&id=1-ZciGB5VJK544sZCafYaukbjsHDCQagH&confirm=t&uuid=80b542f0-ffff-4e57-86af-dd4827648b72
To: /content/data.tar
100%|██████████| 4.34G/4.34G [00:35<00:00, 124MB/s]


Files extracted to 'data':
data.tar deleted successfully.


### Extract Data from the json file

In [5]:
def read_data_in_chunks(file, cols=None, chunk_size=500000, nr_chunks=-1, criteria=None):
    """
    Reads large JSON files in chunks to avoid memory overload. Allows filtering columns
    and applying criteria for filtering rows.

    Parameters:
    file (str): Path to the JSON file.
    cols (list): List of columns to be extracted from each chunk. If None, all columns are read.
    chunk_size (int): Number of rows per chunk.
    nr_chunks (int): Number of chunks to read. If -1, reads until the file ends.
    criteria (function): A function to filter rows in the DataFrame.

    Returns:
    pd.DataFrame: A concatenated DataFrame from all chunks.
    """

    print(f'Reading JSON file in chunks from {file}...')

    try:
        # Initialize the JSON reader
        reader = pd.read_json(path_or_buf=file, chunksize=chunk_size, lines=True)
    except ValueError as e:
        print(f'Error! Failed to read file {file}. Reason: {e}')
        return None

    chunk_list = []
    chunks_read = 0

    while nr_chunks != 0:
        try:
            # Read next chunk
            chunk = next(reader)
            chunks_read += 1

            # Extract specific columns if provided
            if cols is not None:
                chunk = chunk[cols]

            # Apply criteria to filter rows
            if criteria is not None:
                chunk = criteria(chunk)

            chunk_list.append(chunk)
            print(f'Chunk {chunks_read} read: Shape {chunk.shape}')

        except StopIteration:
            print('Info: No more chunks to read.')
            break
        except Exception as e:
            print(f'Error while processing chunk {chunks_read}: {e}')
            print(traceback.format_exc())
            break

        nr_chunks -= 1

    if chunk_list:
        try:
            # Concatenate all chunks into a single DataFrame
            result_df = pd.concat(chunk_list, ignore_index=True, join='outer', axis=0)
            print(f'Concatenation complete. Final DataFrame shape: {result_df.shape}')
            return result_df
        except Exception as e:
            print(f'Error while concatenating chunks: {e}')
            print(traceback.format_exc())
            return None
    else:
        print('Warning: No chunks were processed.')
        return None

In [6]:
base_wdir = '/content/data/'

In [8]:
business_path = base_wdir + 'yelp_academic_dataset_business.json'

business_dataframe = read_data_in_chunks(business_path)

print(f'Shape: {business_dataframe.shape}')
business_dataframe.head()

Reading JSON file in chunks from /content/data/yelp_academic_dataset_business.json...
Chunk 1 read: Shape (150346, 14)
Info: No more chunks to read.
Concatenation complete. Final DataFrame shape: (150346, 14)
Shape: (150346, 14)


business_id                      name  \
0  Pns2l4eNsfO8kk83dixA6A  Abby Rappoport, LAC, CMQ   
1  mpf3x-BjTdTEA3yCZrAYPw             The UPS Store   
2  tUFrWirKiKi_TAnsVWINQQ                    Target   
3  MTSW4McQd7CbVtyjqoe9mw        St Honore Pastries   
4  mWMc6_wTdE0EUBKIGXDVfA  Perkiomen Valley Brewery   

                           address           city state postal_code  \
0           1616 Chapala St, Ste 2  Santa Barbara    CA       93101   
1  87 Grasso Plaza Shopping Center         Affton    MO       63123   
2             5255 E Broadway Blvd         Tucson    AZ       85711   
3                      935 Race St   Philadelphia    PA       19107   
4                    101 Walnut St     Green Lane    PA       18054   

    latitude   longitude  stars  review_count  is_open  \
0  34.426679 -119.711197    5.0             7        0   
1  38.551126  -90.335695    3.0            15        1   
2  32.223236 -110.880452    3.5            22        0   
3  39.955505  -75.155564    4.0            80        1   
4  40.338183  -75.471659    4.5            13        1   

                                          attributes  \
0                      {'ByAppointmentOnly': 'True'}   
1             {'BusinessAcceptsCreditCards': 'True'}   
2  {'BikeParking': 'True', 'BusinessAcceptsCredit...   
3  {'RestaurantsDelivery': 'False', 'OutdoorSeati...   
4  {'BusinessAcceptsCreditCards': 'True', 'Wheelc...   

                                          categories  \
0  Doctors, Traditional Chinese Medicine, Naturop...   
1  Shipping Centers, Local Services, Notaries, Ma...   
2  Department Stores, Shopping, Fashion, Home & G...   
3  Restaurants, Food, Bubble Tea, Coffee & Tea, B...   
4                          Brewpubs, Breweries, Food   

                                               hours  
0                                               None  
1  {'Monday': '0:0-0:0', 'Tuesday': '8:0-18:30', ...  
2  {'Monday': '8:0-22:0', 'Tuesday': '8:0-22:0', ...  
3  {'Monday': '7:0-20:0', 'Tuesday': '7:0-20:0', ...  
4  {'Wednesday': '14:0-22:0', 'Thursday': '16:0-2...

In [9]:
checkin_path = base_wdir + 'yelp_academic_dataset_checkin.json'

checkin_dataframe = read_data_in_chunks(checkin_path)

print(f'Shape: {checkin_dataframe.shape}')
checkin_dataframe.head()


Reading JSON file in chunks from /content/data/yelp_academic_dataset_checkin.json...
Chunk 1 read: Shape (131930, 2)
Info: No more chunks to read.
Concatenation complete. Final DataFrame shape: (131930, 2)
Shape: (131930, 2)


business_id                                               date
0  ---kPU91CF4Lq2-WlRu9Lw  2020-03-13 21:10:56, 2020-06-02 22:18:06, 2020...
1  --0iUa4sNDFiZFrAdIWhZQ  2010-09-13 21:43:09, 2011-05-04 23:08:15, 2011...
2  --30_8IhuyMHbSOcNWd6DQ           2013-06-14 23:29:17, 2014-08-13 23:20:22
3  --7PUidqRWpRSpXebiyxTg  2011-02-15 17:12:00, 2011-07-28 02:46:10, 2012...
4  --7jw19RH9JKXgFohspgQw  2014-04-21 20:42:11, 2014-04-28 21:04:46, 2014...

In [10]:
# Get the business IDs for businesses located in Philadelphia
philadelphia_business_ids = business_dataframe[business_dataframe['city'] == 'Philadelphia']['business_id']

# Filter the checkin dataframe to only include check-ins for businesses in Philadelphia
checkin_dataframe= checkin_dataframe[checkin_dataframe['business_id'].isin(philadelphia_business_ids)]


In [11]:
checkin_dataframe

business_id  \
14      --OS_I7dnABrXvRCCuWOGQ   
32      --sXnWH9Xm6_NvIjyuA99w   
48      -0M0b-XhtFagyLmsBtOe8w   
49      -0PN_KFPtbnLQZEeb23XiA   
50      -0TffRSXXIlBYVbb5AwfTg   
...                        ...   
131890  zyge4T5eSiPHq1-IaJb_Qg   
131891  zyghhZzPgb1bRAIYB-oi1w   
131902  zz-fcqurtm77bZ_rVvo2Lw   
131906  zz3E7kmJI2r2JseE6LAnrw   
131929  zzyx5x0Z7xXWWvWnZFuxlQ   

                                                     date  
14                                    2018-05-11 18:23:36  
32      2011-06-08 19:26:16, 2011-07-31 02:53:37, 2011...  
48      2012-02-09 22:11:59, 2012-02-17 23:03:05, 2012...  
49      2011-02-23 05:39:14, 2011-03-01 21:58:33, 2011...  
50      2013-05-31 23:03:18, 2013-05-31 23:05:01, 2013...  
...                                                   ...  
131890  2010-08-03 21:24:56, 2010-08-07 19:18:20, 2010...  
131891  2015-04-29 16:52:39, 2015-06-29 16:19:11, 2015...  
131902  2011-11-30 19:57:40, 2012-02-15 21:19:38, 2012...  
131906  2010-09-11 14:19:48, 2010-09-25 16:15:17, 2010...  
131929                                2015-01-06 17:51:53  

[12892 rows x 2 columns]

In [12]:
checkin_dataframe.to_csv('/content/data/checkin_dataframe.csv', index=False)

In [7]:
review_path = base_wdir + 'yelp_academic_dataset_review.json'

review_dataframe = read_data_in_chunks(review_path)

print(f'Shape: {review_dataframe.shape}')
review_dataframe.head()

Reading JSON file in chunks from /content/data/yelp_academic_dataset_review.json...
Chunk 1 read: Shape (500000, 9)
Chunk 2 read: Shape (500000, 9)
Chunk 3 read: Shape (500000, 9)
Chunk 4 read: Shape (500000, 9)
Chunk 5 read: Shape (500000, 9)
Chunk 6 read: Shape (500000, 9)
Chunk 7 read: Shape (500000, 9)
Chunk 8 read: Shape (500000, 9)
Chunk 9 read: Shape (500000, 9)
Chunk 10 read: Shape (500000, 9)
Chunk 11 read: Shape (500000, 9)
Chunk 12 read: Shape (500000, 9)
Chunk 13 read: Shape (500000, 9)
Chunk 14 read: Shape (490280, 9)
Info: No more chunks to read.
Concatenation complete. Final DataFrame shape: (6990280, 9)
Shape: (6990280, 9)


review_id                 user_id             business_id  \
0  KU_O5udG6zpxOg-VcAEodg  mh_-eMZ6K5RLWhZyISBhwA  XQfwVwDr-v0ZS3_CbbE5Xw   
1  BiTunyQ73aT9WBnpR9DZGw  OyoGAe7OKpv6SyGZT5g77Q  7ATYjTIgM3jUlt4UM3IypQ   
2  saUsX_uimxRlCVr67Z4Jig  8g_iMtfSiwikVnbP2etR0A  YjUWPpI6HXG530lwP-fb2A   
3  AqPFMleE6RsU23_auESxiA  _7bHUi9Uuf5__HHc_Q8guQ  kxX2SOes4o-D3ZQBkiMRfA   
4  Sx8TMOWLNuJBWer-0pcmoA  bcjbaE6dDog4jkNY91ncLQ  e4Vwtrqf-wpJfwesgvdgxQ   

   stars  useful  funny  cool  \
0      3       0      0     0   
1      5       1      0     1   
2      3       0      0     0   
3      5       1      0     1   
4      4       1      0     1   

                                                text                date  
0  If you decide to eat here, just be aware it is... 2018-07-07 22:09:11  
1  I've taken a lot of spin classes over the year... 2012-01-03 15:28:18  
2  Family diner. Had the buffet. Eclectic assortm... 2014-02-05 20:30:30  
3  Wow!  Yummy, different,  delicious.   Our favo... 2015-01-04 00:01:03  
4  Cute interior and owner (?) gave us tour of up... 2017-01-14 20:54:15

In [11]:
tip_path = base_wdir + 'yelp_academic_dataset_tip.json'

tip_dataframe = read_data_in_chunks(tip_path)

print(f'Shape: {tip_dataframe.shape}')
tip_dataframe.head()

Reading JSON file in chunks from /content/data/yelp_academic_dataset_tip.json...
Chunk 1 read: Shape (500000, 5)
Chunk 2 read: Shape (408915, 5)
Info: No more chunks to read.
Concatenation complete. Final DataFrame shape: (908915, 5)
Shape: (908915, 5)


user_id             business_id  \
0  AGNUgVwnZUey3gcPCJ76iw  3uLgwr0qeCNMjKenHJwPGQ   
1  NBN4MgHP9D3cw--SnauTkA  QoezRbYQncpRqyrLH6Iqjg   
2  -copOvldyKh1qr-vzkDEvw  MYoRNLb5chwjQe3c_k37Gg   
3  FjMQVZjSqY8syIO-53KFKw  hV-bABTK-glh5wj31ps_Jw   
4  ld0AperBXk1h6UbqmM80zw  _uN0OudeJ3Zl_tf6nxg5ww   

                                                text                date  \
0                     Avengers time with the ladies. 2012-05-18 02:17:21   
1  They have lots of good deserts and tasty cuban... 2013-02-05 18:35:10   
2             It's open even when you think it isn't 2013-08-18 00:56:08   
3                          Very decent fried chicken 2017-06-27 23:05:38   
4             Appetizers.. platter special for lunch 2012-10-06 19:43:09   

   compliment_count  
0                 0  
1                 0  
2                 0  
3                 0  
4                 0

In [ ]:
user_path = base_wdir + 'yelp_academic_dataset_user.json'

user_dataframe = read_data_in_chunks(user_path)

print(f'Shape: {user_dataframe.shape}')
user_dataframe.head()

Reading JSON file in chunks from /content/data/yelp_academic_dataset_user.json...
